In [40]:
import quantities as units
from quantities import UnitQuantity, C, N, m, A, mm, V, F, \
  pF, H, ohm, Hz, MHz, S, s, kg, mS, rad
from numpy import pi, log10, e, real, sqrt, imag
from sympy import latex, lambdify, symbols

def Units(x, units):
  x.units = units
  return x

In [36]:
epsilon0 = 8.854187817 * 10**(-12) * C**2/(N*m**2)
mu0 = 4*pi*10**(-7) * N/(A**2)

Np = UnitQuantity('Np', 1) #neper is a dimensionless unit that functions like the decible but with base e instead of base 10
dB = UnitQuantity('dB', Np/log10(e))
nH = UnitQuantity('nH', H*1e-9)

# 1

Transmission line of parallel brass strips with $\sigma_\text{conductor} =1.6\cdot 10^7 \frac{\text{S}}{\text{m}}, w_\text{conductor} = 20\text{ mm}, \epsilon_r=3, \mu = \mu_0$,
$\sigma_\text{dielectric} = 10^{-3}\frac{\text{S}}{\text{m}}, d_\text{dieletric} = 2.5 \text{ mm}, \text{ and}, f = 500\text{ MHz}$.
1. Find $R, L, G$, and $C$
2. Compare magnitiudes of the axial and transmission components of $E$ field.
3. Find $\gamma$ and $Z_0$.

# For low-loss parallel plate waveguide:

In [43]:
#for parallel plate waveguide:
def capacitance_per_length(epsilon_r, w, d, units = pF / m):
  c = epsilon0 * epsilon_r * w / d
  c.units = units
  return c

def inductance_per_length(mu_r, d, w, units = nH/m):
  L = mu0*mu_r*d/w
  L.units = units
  return L

def intrinsic_impedance_per_length(f, mu_r, sigma): #also called wave impedance and is defined as the ratio of E field over H field
  #times two because there are two plates
  return Units((2/w)*(1+1j)*((pi*f*mu_r*mu0)/sigma)**0.5, units = ohm/m)

def conductance_per_length(sigma_dieletric, width_conductor, distance_dielectric):
  return sigma_dieletric*width_conductor/distance_dielectric

def loss_tangent(omega, epsilon, sigma): # eta
  return (omega*imag(epsilon)+sigma)/(omega*real(epsilon))

def Alpha(R, c, L, G):
  return Units(0.5*(R*sqrt(c/L)+G*sqrt(L/c)), units = Np/m)

def Beta(omega, L, capacitance):
  return Units(omega*sqrt(L * capacitance), units = rad/m)

def Gamma(alpha = None, beta = None, omega = None, R = None, C = None, L = None, G = None):
  if alpha is not None and beta is not None:
    return alpha + j*beta
  else:
    return sqrt((R + j*omega*L)*(G + j*omega*C))

In [51]:
epsilon_r = 3; w = 20 * mm; d = 2.5 * mm; mu_r = 1; f = 500*MHz;
sigma_conductor = 1.6e7 * S / m; sigma_dieletric = 1e-3 * S/m

capacitance = capacitance_per_length(epsilon_r, w, d)
print(capacitance)
L = inductance_per_length(mu_r, d, w)
print(L)
R = real(intrinsic_impedance_per_length(f, mu_r, sigma_conductor))
print(R)
G = Units(conductance_per_length(sigma_dieletric, w, d), units = mS/m)
print(G)
alpha = Alpha(R, capacitance, L, G)
print(alpha)
beta = Beta(omega = 2*pi*f, L = L, capacitance = capacitance)
print(beta)
gamma = Gamma(alpha, beta)
print(gamma.simplified)

212.50050760799994 pF/m
157.07963267948963 nH/m
1.1107207345395913 ohm/m
8.0 mS/m
0.12917921969133273 Np/m
18.150550313417227 rad/m
(0.12917921969133273+18.150550313417227j) 1/m


# 2

## For lossy transmission line with parameters:

$f = 100\text{MHz}$
<br></br>
$Z_0 = 50 \Omega$
<br></br>
$\alpha = 0.01 \frac{\text{dB}}{\text{m}}$
<br></br>
$\beta = 0.8\pi\frac{\text{rad}}{\text{m}}$
<br></br>
Find $R, L, G,$ and $C$.

In [ ]:
j = 1j
def admittance_per_unit_length(G, C, omega):
  return G + j*omega*C

def characteristic_impedance_per_unit_length(R, L, omega):
  return R + j*omega*L

In general, $Z_0 = \sqrt{\frac{Z}{Y}}$, where $Z$ is the characteristic impedance and $Y$ is the characteristic admittance.  But when the reactance is zero, we have that
$\frac{Z}{Y} = \frac{R}{G} = \frac{L}{C} \implies Z_0 = \sqrt{\frac{L}{C}}$.
Also $\alpha = R\sqrt{\frac{C}{L}}$.
$\beta$ is the same as for low-loss line, $\beta = \omega \sqrt{LC}$


# 3
## Low-loss Uniform 2 wire transmission line:
$\epsilon_r = 2.25$,
a. Assuming $Z_0 = 300\Omega$ and $r = 0.6 \text{mm}$, find the distance of separation between the lines, call it $d$.
## Coaxial cable
b. For a coaxial line with $Z_0 = 75\Omega$ and $r_\text{center} = 0.6\text{mm}$, find $r_\text{outer}$